# Simple retrieval augmented generation with OpenAI

<!-- TABS -->
## Connect to superduper

:::note
Note that this is only relevant if you are running superduper in development mode.
Otherwise refer to "Configuring your production system".
:::

In [5]:
APPLY = True
SAMPLE_COLLECTION_NAME = 'sample_simple_rag'
COLLECTION_NAME = '<var:table_name>' if not APPLY else 'docs'
ID_FIELD = '<var:id_field>' if not APPLY else 'id'
OUTPUT_PREFIX = 'outputs__'
EAGER = True

In [2]:
from superduper import superduper, CFG
import os

db = superduper('mongomock://', initialize_cluster=False)

2025-Mar-19 11:09:23.33| INFO     | Duncans-MacBook-Pro.local| superduper.misc.importing:13   | Loading plugin: mongodb
2025-Mar-19 11:09:23.38| INFO     | Duncans-MacBook-Pro.local| superduper.base.artifacts:178  | Creating artifact store directory
2025-Mar-19 11:09:23.41| INFO     | Duncans-MacBook-Pro.local| superduper.base.datalayer:51   | Building Data Layer
2025-Mar-19 11:09:23.41| INFO     | Duncans-MacBook-Pro.local| superduper.base.datalayer:68   | Data Layer built
2025-Mar-19 11:09:23.41| INFO     | Duncans-MacBook-Pro.local| superduper.base.build:148  | Configuration: 
 +----------------+-------------------------------+
| Configuration  |             Value             |
+----------------+-------------------------------+
|  Data Backend  |          mongomock://         |
| Artifact Store | filesystem://./artifact_store |
|     Cache      |           in-process          |
+----------------+-------------------------------+


In [ ]:
db.drop(True, True)
db = superduper('mongomock://')

In [6]:
import json
import requests
import io
from superduper import logging


def getter():
    logging.info('Downloading data...')
    response = requests.get('https://superduperdb-public-demo.s3.amazonaws.com/text.json')
    logging.info('Downloading data... (Done)')
    data = json.loads(response.content.decode('utf-8'))
    return [{'x': r} for r in data]

In [7]:
if APPLY:
    data = getter()

2025-Mar-19 11:09:46.91| INFO     | Duncans-MacBook-Pro.local| __main__:8    | Downloading data...
2025-Mar-19 11:09:47.91| INFO     | Duncans-MacBook-Pro.local| __main__:10   | Downloading data... (Done)


<!-- TABS -->
## Insert simple data

After turning on auto_schema, we can directly insert data, and superduper will automatically analyze the data type, and match the construction of the table and datatype.

In [8]:
if APPLY:
    from superduper import Document, Table
    table = Table(COLLECTION_NAME, fields={'x': 'str'})
    db.apply(table, force=True)
    ids = db[COLLECTION_NAME].insert(data)

2025-Mar-19 11:09:53.21| INFO     | Duncans-MacBook-Pro.local| superduper.base.datalayer:388  | Load (('Table', 'docs')) from metadata...
2025-Mar-19 11:09:53.21| INFO     | Duncans-MacBook-Pro.local| superduper.base.apply:94   | Found these changes and/ or additions that need to be made:
2025-Mar-19 11:09:53.21| INFO     | Duncans-MacBook-Pro.local| superduper.base.apply:96   | ----------------------------------------------------------------------------------------------------
2025-Mar-19 11:09:53.21| INFO     | Duncans-MacBook-Pro.local| superduper.base.apply:97   | METADATA EVENTS:
2025-Mar-19 11:09:53.21| INFO     | Duncans-MacBook-Pro.local| superduper.base.apply:98   | ----------------------------------------------------------------------------------------------------
2025-Mar-19 11:09:53.21| INFO     | Duncans-MacBook-Pro.local| superduper.base.apply:109  | [0]: Table:docs:9d5ece60e98371942224b00c3e86aa35: create
2025-Mar-19 11:09:53.21| INFO     | Duncans-MacBook-Pro.local| sup

Create plugin:

In [9]:
from superduper import Plugin

plugin = Plugin(path='./rag_plugin.py')

2025-Mar-19 11:10:06.22| INFO     | Duncans-MacBook-Pro.local| superduper.components.plugin:113  | Preparing plugin 
2025-Mar-19 11:10:06.22| INFO     | Duncans-MacBook-Pro.local| superduper.components.plugin:119  | Downloading plugin  to ./rag_plugin.py
2025-Mar-19 11:10:06.22| INFO     | Duncans-MacBook-Pro.local| superduper.components.plugin:124  | Copying plugin [] to /Users/dodo/.superduper/plugins/2c995a61d2159af5626eb889c73b73b4/rag_plugin.py


<!-- TABS -->
## Apply a chunker for search

:::note
Note that applying a chunker is ***not*** mandatory for search.
If your data is already chunked (e.g. short text snippets or audio) or if you
are searching through something like images, which can't be chunked, then this
won't be necessary.
:::

In [10]:
from superduper import Listener
from rag_plugin import Chunker

upstream_listener = Listener(
    model=Chunker(identifier='chunker'),
    select=db[COLLECTION_NAME],
    key='x',
    identifier='chunker',
    flatten=True,
    upstream=[plugin],
)

In [11]:
if APPLY and EAGER:
    db.apply(upstream_listener, force=True)

2025-Mar-19 11:10:23.83| INFO     | Duncans-MacBook-Pro.local| superduper.base.datalayer:388  | Load (('Listener', 'chunker')) from metadata...
2025-Mar-19 11:10:23.84| INFO     | Duncans-MacBook-Pro.local| superduper.base.datalayer:388  | Load (('Plugin', 'plugin-rag_plugin_py')) from metadata...
2025-Mar-19 11:10:23.84| INFO     | Duncans-MacBook-Pro.local| superduper.base.artifacts:273  | Copying file /Users/dodo/.superduper/plugins/2c995a61d2159af5626eb889c73b73b4/rag_plugin.py to ./artifact_store/a2c558ded9e863b4bbc199b5d9d80b3b824e532bdaee981282c6e7435afa868d/rag_plugin.py
2025-Mar-19 11:10:23.84| INFO     | Duncans-MacBook-Pro.local| superduper.base.datalayer:388  | Load (('Chunker', 'chunker')) from metadata...
2025-Mar-19 11:10:23.85| INFO     | Duncans-MacBook-Pro.local| superduper.base.datalayer:388  | Load (('Table', '_outputs__chunker__56aab886d6081f062dbd14e1e346483f')) from metadata...
2025-Mar-19 11:10:23.85| INFO     | Duncans-MacBook-Pro.local| superduper.base.apply:9

## Select outputs of upstream listener

:::note
This is useful if you have performed a first step, such as pre-computing 
features, or chunking your data. You can use this query to 
operate on those outputs.
:::

<!-- TABS -->
## Build text embedding model

OpenAI:

In [12]:
import os

from superduper_openai import OpenAIEmbedding

openai_embedding = OpenAIEmbedding(
    identifier='text-embedding',
    model='text-embedding-ada-002',
    datatype='vector[float:1536]',
)

## Create vector-index

In [13]:
from superduper import VectorIndex, Listener

vector_index_name = 'vectorindex'

vector_index = VectorIndex(
    vector_index_name,
    indexing_listener=Listener(
        key=upstream_listener.outputs,
        select=db[upstream_listener.outputs],
        model=openai_embedding,
        identifier='embeddinglistener',
        upstream=[upstream_listener],
    )
)

In [14]:
if APPLY and EAGER:
    db.apply(vector_index, force=True)

2025-Mar-19 11:10:46.60| INFO     | Duncans-MacBook-Pro.local| superduper.base.datalayer:388  | Load (('VectorIndex', 'vectorindex')) from metadata...
2025-Mar-19 11:10:46.61| INFO     | Duncans-MacBook-Pro.local| superduper.base.datalayer:388  | Load (('Listener', 'embeddinglistener')) from metadata...
2025-Mar-19 11:10:46.61| INFO     | Duncans-MacBook-Pro.local| superduper.base.datalayer:388  | Load (('Listener', 'chunker')) from metadata...
2025-Mar-19 11:10:46.61| INFO     | Duncans-MacBook-Pro.local| superduper.base.artifacts:286  | Loading file a2c558ded9e863b4bbc199b5d9d80b3b824e532bdaee981282c6e7435afa868d from ./artifact_store
2025-Mar-19 11:10:46.61| INFO     | Duncans-MacBook-Pro.local| superduper.components.plugin:105  | Plugin ./artifact_store/a2c558ded9e863b4bbc199b5d9d80b3b824e532bdaee981282c6e7435afa868d/rag_plugin.py already exists in cache
2025-Mar-19 11:10:46.61| INFO     | Duncans-MacBook-Pro.local| superduper.base.datalayer:388  | Load (('Plugin', 'plugin-rag_plug

[2025-03-19 11:10:47] httpx INFO HTTP Request: GET https://api.openai.com/v1/models "HTTP/1.1 200 OK"
[2025-03-19 11:10:48] httpx INFO HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"                                                                                                                     | 0/4 [00:00<?, ?it/s]
[2025-03-19 11:10:50] httpx INFO HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"                                                                                                             | 1/4 [00:01<00:04,  1.64s/it]
[2025-03-19 11:10:52] httpx INFO HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"████                                                                                                         | 2/4 [00:03<00:04,  2.04s/it]
[2025-03-19 11:10:54] httpx INFO HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"██████████████████████████████████████████████████████

2025-Mar-19 11:10:55.39| INFO     | Duncans-MacBook-Pro.local| superduper.base.datalayer:388  | Load (('Table', '_outputs__embeddinglistener__c01e75d95e8c6cbf8bbb4b429ba0e3c6')) from metadata...
2025-Mar-19 11:10:55.56| INFO     | Duncans-MacBook-Pro.local| superduper.base.datalayer:388  | Load (('Table', 'Job')) from metadata...
2025-Mar-19 11:10:55.57| INFO     | Duncans-MacBook-Pro.local| superduper.base.artifacts:286  | Loading file a2c558ded9e863b4bbc199b5d9d80b3b824e532bdaee981282c6e7435afa868d from ./artifact_store
2025-Mar-19 11:10:55.57| INFO     | Duncans-MacBook-Pro.local| superduper.components.plugin:105  | Plugin ./artifact_store/a2c558ded9e863b4bbc199b5d9d80b3b824e532bdaee981282c6e7435afa868d/rag_plugin.py already exists in cache
2025-Mar-19 11:10:55.60| INFO     | Duncans-MacBook-Pro.local| superduper.base.datalayer:388  | Load (('Table', 'Job')) from metadata...
2025-Mar-19 11:10:55.60| INFO     | Duncans-MacBook-Pro.local| superduper.base.artifacts:286  | Loading file 

<!-- TABS -->
## Build LLM

In [15]:
from superduper_openai import OpenAIChatCompletion


llm_openai = OpenAIChatCompletion(
    identifier='llm-model',
    model='gpt-3.5-turbo',
    datatype='str',
)

## Answer question with LLM

In [ ]:
from superduper import model
from rag_plugin import RAGModel


prompt_template = (
    "Use the following context snippets, these snippets are not ordered!, Answer the question based on this context.\n"
    "These snippets are samples from our internal data-repositories, and should be used exclusively and as a matter"
    " of priority to answer the question. Please answer in 20 words or less.\n\n"
    "{context}\n\n"
    "Here's the question: {query}"
)

rag = RAGModel(
    'simple_rag',
    select=db[upstream_listener.outputs].select().like({upstream_listener.outputs: '<var:query>'}, vector_index=vector_index_name, n=5),
    prompt_template=prompt_template,
    key=upstream_listener.outputs,
    llm=llm_openai,
)

In [ ]:
if APPLY and EAGER:
    db.apply(rag, force=True)

By applying the RAG model to the database, it will subsequently be accessible for use in other services.

In [ ]:
from superduper import Streamlit, Plugin
from rag_plugin import demo_func

demo = Streamlit('simple-rag-demo', demo_func=demo_func)

In [ ]:
from superduper import Application

app = Application(
    'simple-rag-app',
    upstream=[plugin],
    components=[
        upstream_listener,
        vector_index,
        rag,
        demo,
    ]
)

In [ ]:
if APPLY:
    db.apply(app, force=True)

In [ ]:
if APPLY:
    rag = db.load('RAGModel', 'simple_rag')
    print(rag.predict('Tell me about vector-search in the project and the use of lance.'))


You can now load the model elsewhere and make predictions using the following command.

## Create template

In [ ]:
from superduper import Template, Table, Schema
from superduper.components.dataset import RemoteData

template = Template(
    'simple_rag',
    template=app,
    substitutions={
        COLLECTION_NAME: 'table_name',
        'text-embedding-ada-002': 'embedding_model',
        'gpt-3.5-turbo': 'llm_model',
    },
    template_variables=['table_name', 'id_field', 'embedding_model', 'llm_model'],
    default_tables=[
        Table(
            'sample_simple_rag',
            fields={'x': 'str'},
            data=RemoteData(
                'superduper-docs',
                getter=getter,
            )
        ),
    ],
    types={
        'id_field': {
            'type': 'str',
            'default': '_id',
        },
        'embedding_model': {
            'type': 'str',
            'default': 'text-embedding-ada-002',
            'choices': ['text-embedding-ada-002', 'nomic-embed-text:latest'],
        },
        'llm_model': {
            'type': 'str',
            'default': 'gpt-3.5-turbo',
            'choices': ['gpt-3.5-turbo', 'gpt-4-turbo', 'llama3.1:8b']
        },
        'table_name': {
            'type': 'str',
            'default': SAMPLE_COLLECTION_NAME,
        }
    },
    schema={
        "id_field": "id_field",
        "embedding_model": "embedding_model",
        "llm_model": "llm_model",
        "table_name": "table_name",
    },
    db=db
)